This idea is to calculate the probability of different states with a Markov Chain analysis. 

This will focus on the world's greatest game: Snakes and Ladders.


In [1]:
function displayTransitionMatrix (transitionMatrix)
    rowCount = size(transitionMatrix)(1);
    columnCount = size(transitionMatrix)(2);
    header = "{i,j}";
    for i =1:columnCount
        if i < 10
            header = sprintf("%s    [%d]", header, i);
        else
            header = sprintf("%s   [%d]", header, i);
        end
    end
    disp(header)
    for i =1:rowCount
        if i < 10
            rowString = sprintf("  [%d]", i);
        else
            rowString = sprintf(" [%d]", i);
        end
        for j = 1:columnCount
            cell = transitionMatrix(i,j);
            if j < columnCount
                delimeter = "|";
            else
                delimeter = "";
            end
            if cell == 0 || cell == 1
                cellStr = sprintf("     %d%s", cell, delimeter);
            else
                cellStr = sprintf(" %.03f%s", cell, delimeter);
            end
            rowString = sprintf("%s%s", rowString, cellStr);
        end
        disp(rowString);
    end
end

In [2]:
dieMax = 6; # largest you can roll. The smallest you can roll is 1

possibleStateNumber = 16; # This should be a square of some number, like 16,25,36, etc... because the board is square
transitionMatrix = zeros(possibleStateNumber, possibleStateNumber);

In [3]:
# Because players can roll higher than the available spaces then we need to catch those extras and increase the value of the final score
# Otherwise, all possibilities should have a score equal to 1/6, or however many sides the die has

for i =1:possibleStateNumber
    overshootCount = 0;
    for j =1+i:dieMax+i
        if j > possibleStateNumber
            overshootCount = overshootCount + 1;
        else
            transitionMatrix(i,j) = 1/dieMax;
        end
    end
    if overshootCount > 0
        transitionMatrix(i,possibleStateNumber) = transitionMatrix(i,possibleStateNumber) + overshootCount/dieMax;
    end
end

disp("\nT^1\n")
displayTransitionMatrix(transitionMatrix)


T^1

{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0|     0|     0|     0|     0|     0
  [2]     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0|     0|     0|     0|     0
  [3]     0|     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0|     0|     0|     0
  [4]     0|     0|     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0|     0|     0
  [5]     0|     0|     0|     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0|     0
  [6]     0|     0|     0|     0|     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0|     0
  [7]     0|     0|     0|     0|     0|     0|     0| 0.167| 0.167| 0.167| 0.167| 0.167| 0.167|     0|     0|     0
  [8]     0|     0|     0|     0|     0|     0|     0|   

Let's introduce some snakes and ladders

These have the same structure, in that they take the player from position 1 and take them to position 2, in the form (1,2)

Snakes and ladders can end at the same place, but can't start at the same place. A ladder can't end where a snake starts, and vice-versa.

For every state (row) in the transition matrix, replace the probability of the first index with zero, and add the probability to the second index

In [4]:
snakes = [
    13 11;
    10 6;
    15 2
]

snakes =

   13   11
   10    6
   15    2



In [5]:
ladders = [
    3 6;
    7 11;
    5 12
]

ladders =

    3    6
    7   11
    5   12



In [6]:
# Are snakes and ladders equal?

snakePenalty = 0;
for s =1:length(snakes)
    snakePenalty = snakePenalty + snakes(s,1) - snakes(s,2);
end
snakePenalty

ladderReward = 0;
for l =1:length(ladders)
    ladderReward = ladderReward + ladders(l,2) - ladders(l,1);
end
ladderReward

snakePenalty =  19
ladderReward =  14


In [7]:
# Move probabilities from snake head to tail, ladder bottom to top

for i =1:length(transitionMatrix)
    for s =1:length(snakes)
        head = snakes(s, 1);
        tail = snakes(s, 2);
        probability = transitionMatrix(i, head);
        transitionMatrix(i, head) = 0;
        transitionMatrix(i, tail) = transitionMatrix(i, tail) + probability;
    end
    
    for l =1:length(ladders)
        bottom = ladders(l, 1);
        top = ladders(l, 2);
        probability = transitionMatrix(i, bottom);
        transitionMatrix(i, bottom) = 0;
        transitionMatrix(i, top) = transitionMatrix(i, top) + probability;
    end
end

# Squares that don't have a probability (head of snake, bottom of ladder) are now probability = 0

columnCount = size(transitionMatrix)(2)
for i =1:length(transitionMatrix)
    for s =1:length(snakes)
        head = snakes(s, 1);
        transitionMatrix(head, :) = zeros(1, columnCount);
    end
    
    for l =1:length(ladders)
        bottom = ladders(l, 1);
        transitionMatrix(bottom, :) = zeros(1, columnCount);
    end
end

columnCount =  16


In [8]:
for n =1:5
    fprintf("\nT^%d\n",n)
    displayTransitionMatrix(transitionMatrix^n)
end




T^1
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.167|     0| 0.167|     0| 0.333|     0|     0|     0|     0| 0.167| 0.167|     0|     0|     0|     0
  [2]     0|     0|     0| 0.167|     0| 0.333|     0| 0.167|     0|     0| 0.167| 0.167|     0|     0|     0|     0
  [3]     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0
  [4]     0|     0|     0|     0|     0| 0.333|     0| 0.167| 0.167|     0| 0.167| 0.167|     0|     0|     0|     0
  [5]     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0
  [6]     0|     0|     0|     0|     0| 0.167|     0| 0.167| 0.167|     0| 0.333| 0.167|     0|     0|     0|     0
  [7]     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0
  [8]     0|     0|     0|     0|     0| 0.167|     0|    

In [9]:
initialState = zeros(1,possibleStateNumber);
initialState(1,1) = 1;
displayTransitionMatrix(initialState);


{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     1|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0|     0


In [10]:
# Starting at the square=1, how what's the probability of being at any given square after n turns?

for n =1:1:20
    fprintf("\n\nn = %d\n", n);
    newState = initialState*transitionMatrix^n;
    displayTransitionMatrix(newState);
end




n = 1
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.167|     0| 0.167|     0| 0.333|     0|     0|     0|     0| 0.167| 0.167|     0|     0|     0|     0


n = 2
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.056|     0| 0.028|     0| 0.167|     0| 0.111| 0.083|     0| 0.222| 0.139|     0| 0.056|     0| 0.139


n = 3
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.083|     0| 0.009|     0| 0.088|     0| 0.042| 0.051|     0| 0.194| 0.111|     0| 0.093|     0| 0.329


n = 4
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.075|     0| 0.014|     0| 0.061|     0| 0.030| 0.023|     0| 0.127| 0.078|     0| 0.066|     0| 0.526


n = 5
{i,j}    [1]    [2] 

In [11]:
# Starting at the square=8, how what's the probability of being at any given square after n turns?

initialState = zeros(1,possibleStateNumber);
initialState(1,8) = 1;
displayTransitionMatrix(initialState);

n = 3;

fprintf("\n\nn = %d\n", n);
newState = initialState*transitionMatrix^n;
displayTransitionMatrix(newState);

{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|     0|     0|     0|     0|     0|     0


n = 3
{i,j}    [1]    [2]    [3]    [4]    [5]    [6]    [7]    [8]    [9]   [10]   [11]   [12]   [13]   [14]   [15]   [16]
  [1]     0| 0.074|     0| 0.023|     0| 0.065|     0| 0.032| 0.014|     0| 0.111| 0.074|     0| 0.060|     0| 0.546
